In [76]:
from IPython.display import Image
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()

import warnings
warnings.simplefilter("ignore")

## EmazeMeGroups

In [77]:
groups = pd.read_csv('data/EmazeMeGroups.csv')

In [78]:
groups.head()

GroupID          GroupName                                 Admin  Privecy  \
0        1    BeitarJerusalem  d1f13501-60bf-4dd8-b6e8-c11d2f8eab8c        1   
1        2         BeitarFans  d1f13501-60bf-4dd8-b6e8-c11d2f8eab8c        1   
2        3  newgroupwithdate2                                   NaN        0   
3        4      another-group                                   NaN        0   
4        5                 ""                                   NaN        0   

   Visibility  priority  varified  AssociatedGroupsJson  ModeratorsJson  \
0           0         0         0                   NaN             NaN   
1           0         0         0                   NaN             NaN   
2           0         0         0                   NaN             NaN   
3           0         0         0                   NaN             NaN   
4           0         0         0                   NaN             NaN   

                   DateCreated                  DateUpdated  \
0  2020-04-05 10:27:38.0531290  2020-04-05 10:27:38.0531290   
1  2020-04-05 10:27:38.0531290  2020-04-05 10:27:38.0531290   
2  2020-04-05 12:00:36.2022096  2020-04-05 12:00:36.2022096   
3  2020-04-20 08:57:01.8046276  2020-04-20 08:57:01.8046276   
4  2020-04-20 10:44:39.2241056  2020-04-20 10:44:39.2241056   

                                AboutGroup                       EmazeSite  \
0                  Group of Beitar players  https://emaze.me/diogoverdasca   
1  Group for fans of Beitar Jerusalem f.c.                             NaN   
2                                      NaN                             NaN   
3                                      NaN                             NaN   
4                                      NaN                             NaN   

                                       GroupCoverUrl  \
0  //userscontent2.emaze.com/userinfo//cover/2dc9...   
1  //userscontent2.emaze.com/userinfo//cover/b397...   
2                                                NaN   
3                                                NaN   
4                                                NaN   

                                       GroupPhotoUrl  \
0  //userscontent2.emaze.com/userinfo//image/7c4d...   
1  //userscontent2.emaze.com/userinfo//image/1e5a...   
2                                                NaN   
3                                                NaN   
4                                                NaN   

                                        SiteThumnail  
0  //storage.googleapis.com/userscontent2.emaze.c...  
1                                                NaN  
2                                                NaN  
3                                                NaN  
4                                                NaN

In [79]:
groups = groups.rename(columns = {'GroupID':'EmazeMeGroupID'})

## EmazeMeGroupFollowers

In [80]:
group_f = pd.read_csv('data/EmazeMeGroupFollowers.csv')

In [81]:
group_f.head()

FollowId                            FollowerId  EmazeMeGroupId  \
0         3  240238a0-ce99-4ea7-ac77-eff09fb5eff2               1   
1         4  240238a0-ce99-4ea7-ac77-eff09fb5eff2               2   
2         5  e9477dec-c960-4f11-a563-9781cf8b1c15               1   
3         6  e9477dec-c960-4f11-a563-9781cf8b1c15               2   
4         7  93aff917-ec28-4201-b1d2-788e275671b2               1   

            FollowingSince  
0  0001-01-03 00:00:00.000  
1  0001-01-03 00:00:00.000  
2  0001-01-03 00:00:00.000  
3  0001-01-03 00:00:00.000  
4  0001-01-03 00:00:00.000

In [82]:
#create a values column for the matrix, make everything 1 since already followed
group_f['following'] = 1

In [83]:
#dictionary for groupid to groupname
item_dict = dict(zip(groups['EmazeMeGroupID'], groups['GroupName']))

## Create sparse FollowerId and Quantity matrix

In [84]:
group_matrix_df = group_f.pivot_table(
    values='following', 
    index='FollowerId', 
    columns='EmazeMeGroupId'
)

In [85]:
group_matrix_df.head()

EmazeMeGroupId                         1    2   13   14   15   26  30  31  32  \
FollowerId                                                                      
09d32136-ed59-49fd-b553-5c1448aa92fd  NaN  NaN NaN  NaN  NaN  1.0 NaN NaN NaN   
1c1994c7-9bb0-477d-83dd-dc0d6f822c10  NaN  NaN NaN  NaN  NaN  1.0 NaN NaN NaN   
240238a0-ce99-4ea7-ac77-eff09fb5eff2  1.0  1.0 NaN  1.0  1.0  NaN NaN NaN NaN   
2415e720-8d4c-44c7-9cbf-cb1e76dcf4e8  NaN  NaN NaN  NaN  NaN  1.0 NaN NaN NaN   
24c1d879-cb12-4582-87b1-982c3b9ef4e8  NaN  NaN NaN  NaN  NaN  1.0 NaN NaN NaN   

EmazeMeGroupId                        37  38  
FollowerId                                    
09d32136-ed59-49fd-b553-5c1448aa92fd NaN NaN  
1c1994c7-9bb0-477d-83dd-dc0d6f822c10 NaN NaN  
240238a0-ce99-4ea7-ac77-eff09fb5eff2 NaN NaN  
2415e720-8d4c-44c7-9cbf-cb1e76dcf4e8 NaN NaN  
24c1d879-cb12-4582-87b1-982c3b9ef4e8 NaN NaN

In [86]:
follower_id_list = np.array(group_matrix_df.index.tolist())
group_id_list = np.array(group_matrix_df.columns.tolist())

In [87]:
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import svds

In [88]:
group_mtx = group_matrix_df.fillna(0).values.copy()
group_sparse_mtx = coo_matrix(group_mtx)

## SVD of matrix

In [89]:
U, s, V = svds(group_sparse_mtx, k=1)

## Multiply the decompostion back to get approx matrix

In [90]:
s_diag = np.diag(s)

In [91]:
group_svd = U @ s_diag @ V

In [92]:
group_svd

array([[0.28843212, 0.30581057, 0.15243372, 0.12026393, 0.18426903,
        0.48894413, 0.01737844, 0.01737844, 0.01737844, 0.01737844,
        0.01737844],
       [0.28843212, 0.30581057, 0.15243372, 0.12026393, 0.18426903,
        0.48894413, 0.01737844, 0.01737844, 0.01737844, 0.01737844,
        0.01737844],
       [0.53019506, 0.56214007, 0.2802032 , 0.22106879, 0.33872277,
        0.89877564, 0.031945  , 0.031945  , 0.031945  , 0.031945  ,
        0.031945  ],
       [0.28843212, 0.30581057, 0.15243372, 0.12026393, 0.18426903,
        0.48894413, 0.01737844, 0.01737844, 0.01737844, 0.01737844,
        0.01737844],
       [0.28843212, 0.30581057, 0.15243372, 0.12026393, 0.18426903,
        0.48894413, 0.01737844, 0.01737844, 0.01737844, 0.01737844,
        0.01737844],
       [0.3505486 , 0.37166965, 0.1852617 , 0.14616385, 0.22395303,
        0.59424269, 0.02112105, 0.02112105, 0.02112105, 0.02112105,
        0.02112105],
       [0.28843212, 0.30581057, 0.15243372, 0.12026393, 0.

# Rec Function

In [103]:
col = ["user_id","rec_group_id", "rec_group_name"]
rows = []
def recommend(follower_list, num_results=5):
    for follower_id in follower_list:
        follower_indx = np.where(follower_id_list == follower_id)[0][0]
        indx_sort = group_svd[follower_indx, :].argsort()[::-1]
        never_buy = group_mtx[follower_indx, :][indx_sort] == 0
    
        rec_indx = indx_sort[never_buy]
        rec_ids = group_id_list[rec_indx]
        rec_name = [item_dict[x] for x in rec_ids]
        rows.append([follower_id,rec_ids[:num_results],rec_name[:num_results]])

In [104]:
#recommend function takes in a list of users, and outputs a dataframe with the users and their recommendations
recommend(['09d32136-ed59-49fd-b553-5c1448aa92fd','240238a0-ce99-4ea7-ac77-eff09fb5eff2',
           '24c1d879-cb12-4582-87b1-982c3b9ef4e8','e9477dec-c960-4f11-a563-9781cf8b1c15'])

In [105]:
reco_df = pd.DataFrame(rows, columns=col)

In [106]:
reco_df

user_id          rec_group_id  \
0  09d32136-ed59-49fd-b553-5c1448aa92fd    [2, 1, 15, 13, 14]   
1  240238a0-ce99-4ea7-ac77-eff09fb5eff2  [26, 13, 32, 38, 30]   
2  24c1d879-cb12-4582-87b1-982c3b9ef4e8    [2, 1, 15, 13, 14]   
3  e9477dec-c960-4f11-a563-9781cf8b1c15  [26, 14, 32, 38, 30]   

                                      rec_group_name  
0  [BeitarFans, BeitarJerusalem, Israeli Singers,...  
1  [Soccer Players, Emaze, test1235, tttttttttt, ...  
2  [BeitarFans, BeitarJerusalem, Israeli Singers,...  
3  [Soccer Players, Design and Marketing Team, te...

#### Uncomment line below to export file into csv

In [97]:
#reco_df.to_csv('recommend_groups.csv')